# Семинар 1. Вспомнить всё!

План семинара:

- [ ] Вспоминаем основные понятия из статистики: модели и оценки параметров
- [ ] Вспоминаем методы оценок параметров. Основной -- ML.
- [ ] Задача линейной регрессии
- [ ] Вспоминаем тестирование гипотез

### Основные понятия из статистики

Так как данный курс посвящён временным рядам, следует вообще говоря определить, что это такое. В реальной жизни мы наблюдаем много явлений, протяжённых во времени. Будь то котировки акций, уровень ВВП или количество продаж нового айфона. Притом критически важным является деталь, что мы наблюдаем не случайный процесс, а его некоторую **реализацию**. Это и является временным рядом. Мы наблюдаем некоторый частный случай того, как могли произойти события. Но хотелось бы, конечно, видеть полную картину.

Теперь нашей задачей является восстановление общего закона протекания процесса в зависимости от реализации. Но можем ли мы в принципе быть уверены, что сможем его восстановить? В какой-то мере ответ на это даёт [Теорема Дуба о разложении ~~на плесень и липовый мёд~~](https://wikichi.ru/wiki/Doob_decomposition_theorem). Говоря в очень упрощённых терминах, она говорит о том, что почти любой "хороший" процесс можно разложить на прогнозируемую (детерминированную) и принципиально непрогнозируемую части. Следственно, мы никогда не сможем восстановить процесс идеально. Но тем не менее, часть процессов более склонна к детерминированному поведению, а часть -- менее. Например дневная температура яввляется очень сезонной величиной, то есть имеет паттерн, удобный для прогнозирования. С другой стороны, котировки акций наиболее близки к хаотичному движению и весьма трудно поддаются прогнозированию. Этим занимаются скорее в области количественных финансов. В нашем курсе котировки акций могут быть рассмотрены скорее максимум из-за удобства и качества данных, но не более чем для иллюстрации. Исключение составит семинар про GARCH-модели. Также всюду будем рассматривать только дискретные временные ряды.

Удобным представлением для описания дискретных процессов являются случайные величины. То есть мы предполагаем, для каждой точки времени существует некоторая случайная величина, породившая наблюдение. Следовательно, у каждой такой величины существует некоторое **распределение**. Будем предполагать, что это распределение задаётся параметрической функцией. Её параметры могут меняться с течением времени, но функциональная форма -- нет.  Далее нашей задачей будет подбор таких параметров распределения, которые бы максимально приблизили порождаемый процесс к наблюдаемому временному ряду. Для того, чтобы решать такие задачи, необходимо вспомнить ряд определений и зафиксировать обозначения.

$X - \text{некоторая случайная величина, которую мы хотим оценить}$

$X_1 \cdots X_n - \text{n наблюдений (выборка) о случайной величине, тоже случайные величины}$ 

$x_1 \cdots x_n - \text{реализация выборки, конкретные неслучайные числа}$

$\text{Предполагаем, что }X_i \sim f(\theta), \text{ где } \theta - \text{ некоторый набор параметров }$

Наша цель -- получить такую $\hat{\theta} = g(X_1, \cdots, X_n)$, которая максимально подгонит распределение под данные.




In [4]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import plotly.graph_objects as go

In [32]:
sample = np.random.normal(1, 2, 50)

mu_hat = np.mean(sample)
sigma_hat = np.std(sample)

est_x = np.linspace(-10, 10)
est_y = ss.norm.pdf(est_x, mu_hat, sigma_hat)

fig = go.Figure()

fig.add_trace(go.Histogram(x=sample, nbinsx=20, histnorm="probability density", name="Нормированные частоты"))
fig.add_trace(go.Scatter(x=est_x, y=est_y, name="Оценённая плотность"))

fig.show()

### Методы оценки параметров

1. Метод моментов
    * Идея: Истинные моменты должны равняться выборочным. Если моменты распределений похожи, то и сами они схожи.
    * Необходимо выписать систему моментных тождеств. Моментное тождество - это уравнение, в котором с одной стороны стоит выборочный момент, а с другой -- истинный. Решив эти уравнения относительно параметров, получим оценки. Подобно рассматривать в курсе не будем
2. Метод максимального правдоподобия. 
    * Идея: Плотность - это ненормированная вероятность. Давайте максимизируем вероятность получить нашу выборку.
    
    * Так как наблюдения в выборке по предположению независимы, перемножим плотности в точках отдельных наблюдений и получим вероятность пересечения. Полученную функцию, а чаще - её логарифм, максимизируем по параметрам.

$$L(\theta| X) = \prod_{i=1}^nf(\theta | X_i) \to \max_{\theta}$$

$$l(\theta| X) = \sum_{i=1}^nln(f(\theta | X_i)) \to \max_{\theta}$$

Однако во временных рядах существует ряд проблем. Например, наблюдения не являются независимыми между собой. Каждое последующее наблюдение может зависеть от нескольких предыдущих, но, что логично не зависит от будущих. В таком случае необходимо выписывать более сложную версию правдоподобия. Например, такое:

$$L(\theta| X) = f(\theta, X_2, \cdots, X_n | X_1) = \prod_{i=2}^nf(\theta, X_i | X_{i-1}) \to \max_{\theta}$$

### Свойства оценок параметров

Важно отметить, что в общей постановке оценки являются функциями от выборки, которая является случайной величиной. Следовательно, оценки тоже являются случайной величиной.

1. Оценка $\hat{\theta}$ параметра $\theta$ является несмещённой, если $\mathbb{E}(\hat{\theta}) = \theta$
2. Оценка $\hat{\theta}$ параметра $\theta$ является состоятельной, если $plim(\hat{\theta}) = \theta$ при $n \to \infty$
3. Оценка $\hat{\theta}_1$ более эффективна чем  $\hat{\theta}_2$, если $\mathbb{E}(\hat{\theta}_1 - \theta)^2 < \mathbb{E}(\hat{\theta}_2 - \theta)^2$

### Задача линейной регрессии

Задача регрессии - одна из ключевых в статистике. Мы предполагаем некоторую зависимость между переменными и пытаемся её оценить. Формализуем:

Предполагаем зависимость следующего вида: $Y = f(X, \theta) + \varepsilon$, где $\varepsilon$ (случайная величина) - некоторый шум и невключённые в модель факторы. В самой стандартной постановке в качестве $f$ выбирают линейную функцию.

$$ Y = \beta_0 + \beta_1 X + \varepsilon $$

Для того чтобы оценить беты, необходимо снова прибегнуть к какому-либо методу

1. МНК. Метод наименьших квадратов предлагает просто минимизировать $\sum_{i=1}^n(Y - X\beta)^2$. Эта задача имеет аналитическое решение: $\hat{\beta} = (X'X)^{-1}X'Y$. Из минусов: матрицу дорого обращать.

2. Можно указать явное распределение для $\varepsilon$, выразить эту величину из уравнения, а далее использовать стандартные методы для распределений - метод моментов, максимального правдоподобия, etc

### Тестирование гипотез